<a href="https://colab.research.google.com/github/Anonymous1846/Blood-cancer-subtype-classification-using-DCNN/blob/master/Blood_Cancer_Subtype_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the required dependencies !**

In [2]:
import tensorflow

<module 'tensorflow' from '/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py'>
